# Análise de dados - Xadrez

---

# Dados

- **id**;
- **Rated**: partida competitiva (V/F);
- **created_at**: Horário de início;
- **last_move_at**: Horário de término;
- **turns**: Número de rodadas;
- **victory_status**: Resultado da partida;
- **Winner**: Ganhador;
- **increment_code**: Tempo adicional;
- **white_id** : id do branco;
- **white_rating**: Pontuação do Branco;
- **black_id**: id do preto;
- **black_rating**: Pontuação do preto;
- **moves**: Movimentos em formato de anotação de xadrez;
- **opening_eco** : Código preestabelecido para cada abertura;
- **opening_name**: Nome da abertura;
- **opening_ply**: Número de jogadas na fase de abertura; 

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib as mtl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from pandas_profiling import ProfileReport
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
df = pd.read_csv('../input/chess/games.csv')
df.head()

In [ ]:
report = ProfileReport(df)
report

# Limpeza de dados

### Colunas desnecessárias
- As colunas de id não serão uteis nessa análise, pois não ha necessidade de saber quem é o jogador.
- 'last_move_at', 'increment_code' e 'opening_ply' não são desnecessárias para esta análise.

In [ ]:
df = df.drop(['id', 'black_id', 'white_id', 'created_at', 'last_move_at', 'increment_code', 'opening_ply'], axis=1)

In [ ]:
df.head()

## Novas colunas

### Aberturas

In [ ]:
df['abertura'] = df['opening_name']

#Limpeza da coluna 

def splitAbertura(x):
    df['abertura'] = df['abertura'].str.split(x).str[0]
    return df['abertura']

splitAbertura(':')
splitAbertura('|')
splitAbertura('Game')
splitAbertura('#')
splitAbertura('Opening')
df['abertura'] = df['abertura'].str.strip()

### Média
- média de pontos dos jogadores na partida

In [ ]:
df['media_pontos'] = (df['white_rating'] + df['black_rating']) / 2

### Categoria
- Separa os jogadores com base na pontuação
- Essas classes foram retiradas do site oficial do FIDE (Federação Internacional de Xadrez)

In [ ]:
conditions = [
    (df['media_pontos'] <= 199),
    (df['media_pontos'] <= 399),
    (df['media_pontos'] <= 599),
    (df['media_pontos'] <= 799),
    (df['media_pontos'] <= 999),
    (df['media_pontos'] <= 1199),
    (df['media_pontos'] <= 1399),
    (df['media_pontos'] <= 1599),
    (df['media_pontos'] <= 1799),
    (df['media_pontos'] <= 1999),
    (df['media_pontos'] <= 2199),
    (df['media_pontos'] <= 2399),
    (df['media_pontos'] >= 2400),
]

choices = ['Class J', 'Class I', 'Class H', 'Class G',  'Class F',  'Class E',  'Class D',  
           'Class C', 'Class B',  'Class A', 'Expert','National master', 'Senior master']

df['classe'] = np.select(conditions, choices)

### Traduzir as tabela

#### Status de vitoria

In [ ]:
status_vitoria = {
    'draw': 'Empate', 
    'resign': 'Desistencia',
    'outoftime':'TempoEsgotado',
    'mate' : 'Mate'
}
df['victory_status'] = df['victory_status'].replace(status_vitoria)
df['victory_status'].unique()

#### Ganhador

In [ ]:
ganhador= {
    'draw': 'Empate', 
    'black': 'Preto',
    'white':'branco',
}
df['winner'] = df['winner'].replace(ganhador)
df['winner'].unique()

### Primeira Jogada

In [ ]:
df['primeiraJogada_branco'] = df['moves'].str.split(" ").str[0]
df['primeiraJogada_preto'] = df['moves'].str.split(" ").str[1]

# Análise de dados

## Resultado das partidas 

In [ ]:
def contarResultadoPartida(df):
    resultadoPartida = df.groupby(['classe','victory_status']).agg({'rated': 'count'}).reset_index()
    resultadoPartida.columns = ['Classe','Resultado', 'Quantidade']
    return resultadoPartida
    
resultadoPartidaDf = contarResultadoPartida(df)

fig = px.bar(resultadoPartidaDf, x='Classe',y='Quantidade',color='Resultado')
fig.show()

- Visto que a "classe E" e "Expert" possuem uma quantidade de partidas equivalentes, e os jogadores possuem um nível de performance bem diferente, essas classes serão usadas para análises comparativas.

In [ ]:
expertDf = df[df['classe'] == 'Expert']
classeEDf = df[df['classe'] == 'Class E']

resultadoPartidaExpert = contarResultadoPartida(expertDf)

resultadoPartidaClasseE = contarResultadoPartida(classeEDf)

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(
    go.Pie(labels=resultadoPartidaExpert['Resultado'], values=resultadoPartidaExpert['Quantidade']),
    row=1, col=1)

fig.add_trace(
    go.Pie(labels=resultadoPartidaClasseE['Resultado'], values=resultadoPartidaClasseE['Quantidade']),
    row=1, col=2)

fig.update_traces(textposition='inside', hole=.4, hoverinfo="label+percent")
fig.update_layout(
    title_text="Resultados de partidas de Expert e classe E",
    annotations=[dict(text='Expert', x=0.18, y=0.5, font_size=15, showarrow=False),
                 dict(text='Classe E', x=0.83, y=0.5, font_size=15, showarrow=False)])
fig.show()

## Primeiro movimento

### Branco

In [ ]:
def contarPrimeiraJogada(df,colunaPrimeiraJogada):
    primeiraJogadaDf = df.groupby([colunaPrimeiraJogada]).agg({'rated': 'count'}).reset_index()
    primeiraJogadaDf.columns = ['Jogada', 'Quantidade']
    primeiraJogadaDf = primeiraJogadaDf.sort_values(by=['Quantidade'], ascending=False).head(5)
    return primeiraJogadaDf
    
primeiraJogadaBrancoExpert = contarPrimeiraJogada(expertDf,'primeiraJogada_branco')
primeiraJogadaBrancoClasseE = contarPrimeiraJogada(classeEDf,'primeiraJogada_branco')

In [ ]:
sns.set_style('whitegrid')
font = {'size': 17}
mtl.rc('font', **font)

fig, axes = plt.subplots(1, 2, figsize=(17, 6), sharey=True)
fig.suptitle('Primeira jogada das peças brancas',fontsize=27)

sns.barplot(ax=axes[0], x=primeiraJogadaBrancoExpert['Jogada'], y=primeiraJogadaBrancoExpert['Quantidade'])
axes[0].set_title('Experts')

sns.barplot(ax=axes[1], x=primeiraJogadaBrancoClasseE['Jogada'], y=primeiraJogadaBrancoClasseE['Quantidade'])
axes[1].set_title('Classe E')

### Preto

In [ ]:
primeiraJogadaPretoExpert = contarPrimeiraJogada(expertDf,'primeiraJogada_preto')
primeiraJogadaPretoClasseE = contarPrimeiraJogada(classeEDf,'primeiraJogada_preto')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(17, 6), sharey=True)
fig.suptitle('Primeira jogada das peças pretas',fontsize=25)

sns.barplot(ax=axes[0], x=primeiraJogadaPretoExpert['Jogada'], y=primeiraJogadaPretoExpert['Quantidade'])
axes[0].set_title('Experts')

sns.barplot(ax=axes[1], x=primeiraJogadaPretoClasseE['Jogada'], y=primeiraJogadaPretoClasseE['Quantidade'])
axes[1].set_title('Classe E')

## Taxa de vitória

In [ ]:
def gerarTaxaDeVitoria(df):
    taxaDeVitoria = df.groupby(['winner']).agg({'rated': 'count'}).reset_index()
    taxaDeVitoria.columns = ['ganhador', 'quantidade']
    taxaDeVitoria = taxaDeVitoria.drop(0)
    return taxaDeVitoria
    
taxaVitoriaExpert = gerarTaxaDeVitoria(expertDf)
taxaVitoriaClasseE = gerarTaxaDeVitoria(classeEDf)

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(
    go.Pie(labels=taxaVitoriaExpert['ganhador'], values=taxaVitoriaExpert['quantidade']),
    row=1, col=1)

fig.add_trace(
    go.Pie(labels=taxaVitoriaClasseE['ganhador'], values=taxaVitoriaClasseE['quantidade']),
    row=1, col=2)

fig.update_traces(textposition='inside', hole=.4, hoverinfo="label+percent")
fig.update_layout(
    title_text="Taxa de vitoria por cores",
    annotations=[dict(text='Expert', x=0.19, y=0.5, font_size=12, showarrow=False),
                 dict(text='Classe E', x=0.82, y=0.5, font_size=12, showarrow=False)])
fig.show()

## Abertura

In [ ]:
def calcularAberturasMaisUsadas(df):
    aberturasMaisUsadas = df.groupby(['abertura']).agg({'rated': 'count'}).reset_index()
    aberturasMaisUsadas.columns = ['Abertura', 'Quantidade']
    aberturasMaisUsadas = aberturasMaisUsadas.sort_values(by=['Quantidade'], ascending=False).head(10)
    return aberturasMaisUsadas

aberturasMaisUsadasExpert = calcularAberturasMaisUsadas(expertDf)
aberturasMaisUsadasClasseE = calcularAberturasMaisUsadas(classeEDf)

In [ ]:
fig = px.bar(aberturasMaisUsadasExpert, y='Quantidade', x='Abertura',title='Aberturas mais usadas por Experts')
fig.show()

In [ ]:
fig = px.bar(aberturasMaisUsadasClasseE, y='Quantidade', x='Abertura',title='Aberturas mais usadas por classes E')
fig.show()